In [ ]:
%matplotlib inline


In [ ]:

satellite_name = "aquaris"
base_dir = f"D:/Paper_1/segmentation"
dataset_dir = f"{base_dir}/datasets/{satellite_name}"
results_dir = f"{base_dir}/yolo_results/{satellite_name}_results/{satellite_name}_seg"

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")

model.train(
    data=f"{dataset_dir}/data.yaml",
    epochs=20,
    imgsz=1024,
    batch=16,
    save_period=5,
    project=f"{base_dir}/yolo_results/{satellite_name}",
    name="seg_res",
    device=0
)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

csv_path = f"D:/Paper_1/segmentation/yolo_results/{satellite_name}/seg_res/results.csv"

df = pd.read_csv(csv_path)

fig, axs = plt.subplots(2, 2, figsize=(16, 10))
axs = axs.ravel()

# 1. METRICS: mAP, Precision, Recall
axs[0].plot(df["epoch"], df["metrics/mAP50(B)"], 'o-', label="mAP@0.5")
axs[0].plot(df["epoch"], df["metrics/mAP50-95(B)"], 'o-', label="mAP@0.5:0.95")
axs[0].plot(df["epoch"], df["metrics/precision(B)"], 'o-', label="Precision")
axs[0].plot(df["epoch"], df["metrics/recall(B)"], 'o-', label="Recall")
axs[0].set_title("Validation Metrics")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Value")
axs[0].grid(True)
axs[0].legend()

# 2. TRAIN LOSSES
axs[1].plot(df["epoch"], df["train/box_loss"], 'o-', label="Box Loss")
axs[1].plot(df["epoch"], df["train/cls_loss"], 'o-', label="Class Loss")
axs[1].plot(df["epoch"], df["train/dfl_loss"], 'o-', label="DFL Loss")
axs[1].set_title("Training Losses")
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Loss")
axs[1].grid(True)
axs[1].legend()

# 3. VAL LOSSES
axs[2].plot(df["epoch"], df["val/box_loss"], 'o-', label="Box Loss")
axs[2].plot(df["epoch"], df["val/cls_loss"], 'o-', label="Class Loss")
axs[2].plot(df["epoch"], df["val/dfl_loss"], 'o-', label="DFL Loss")
axs[2].set_title("Validation Losses")
axs[2].set_xlabel("Epoch")
axs[2].set_ylabel("Loss")
axs[2].grid(True)
axs[2].legend()

# 4. LEARNING RATES
axs[3].plot(df["epoch"], df["lr/pg0"], 'o-', label="lr_pg0")
axs[3].plot(df["epoch"], df["lr/pg1"], 'o-', label="lr_pg1")
axs[3].plot(df["epoch"], df["lr/pg2"], 'o-', label="lr_pg2")
axs[3].set_title("Learning Rates")
axs[3].set_xlabel("Epoch")
axs[3].set_ylabel("LR")
axs[3].grid(True)
axs[3].legend()

plt.suptitle("YOLOv8 Segmentation Training Summary", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
from ultralytics import YOLO

weights_path = f"D:/Paper_1/segmentation/yolo_results/{satellite_name}/seg_res/weights/best.pt"
data_yaml_path = f"D:/Paper_1/segmentation/datasets/{satellite_name}/data.yaml"

model = YOLO(weights_path)

metrics = model.val(
    data=data_yaml_path,
    split="test",
    imgsz=1024,
    save=True,
    save_txt=True,
    save_conf=True,
    save_crop=False,
    plots=True,
    device=0
)


In [ ]:
from ultralytics import YOLO

weights_path = f"D:/Paper_1/segmentation/yolo_results/{satellite_name}/seg_res/weights/best.pt"
source_path = f"D:/Paper_1/segmentation/datasets/{satellite_name}/test/images"
project_path = f"D:/Paper_1/segmentation/yolo_results/{satellite_name}"

model = YOLO(weights_path)

results = model.predict(
    source=source_path,
    task="segment",
    save=True,
    conf=0.5,
    show_conf=False,
    boxes=False,
    show_labels=True,
    line_width=1,
    retina_masks=True,
    name="predict_masks",
    project=project_path,
    device=0
)
